In [ ]:
%run ./00_ghcn_setup.ipynb
# Run this cell to start a spark session in this notebook

start_spark(executor_instances=2, executor_cores=1, worker_memory=1, master_memory=1)

### Assignment 1 ###

The code below demonstrates how to explore and load the data provided for the assignment from Azure Blob Storage and how to save any outputs that you generate to a separate user container.

**Key points**

- The data provided for the assignment is stored in Azure Blob Storage and outputs that you generate will be stored in Azure Blob Storage as well. Hadoop and Spark can both interact with Azure Blob Storage similar to how they interact with HDFS, but where the replication and distribution is handled by Azure instead. This makes it possible to read or write data in Azure over HTTPS where the path is prefixed by `wasbs://`.
- There are two containers, one for the data which is read only and one for any outputs that you generate,
  - `wasbs://campus-data@madsstorage002.blob.core.windows.net/`
  - `wasbs://campus-user@madsstorage002.blob.core.windows.net/`
- You can use variable interpolation to insert your global username variable into paths automatically.
  - This works for bash commands as well.

**Q1** First you will investigate the `daily`, `stations`, `states`, `countries`, and `inventory` data provided  in cloud storage in:
 `wasbs://campus-data@madsstorage002.blob.core.windows.net/ghcnd/`  
using the `hdfs` command.

**(a)** How is the data structured?

In [ ]:
# Write your imports here or insert cells below

from pyspark.sql           import functions as F 
from pyspark.sql.types     import *
from pyspark.sql.functions import col
import time

In [ ]:
aDaily         = f'/2025.csv.gz'
prefix         = f'wasbs://{azure_data_container_name}@{azure_account_name}.blob.core.windows.net/ghcnd/'
prefixWrite    = f'wasbs://campus-user@madsstorage002.blob.core.windows.net/dew59/'
#prefixWrite    = "./"
prefixDaily    = f'{prefix}/daily/'
dprintf(DEEBUG,prefix)
dprintf(DEEBUG,prefixWrite)
dprintf(DEEBUG,prefixDaily)
dprintf(DEEBUG,"DEEBUG = TRUE")


In [ ]:
# CELL 1: – Load processed DataFrames from Parquet
# CELL 1:
# ------------------------------------------------
# Summary:
# - load analysis outputs saved to prefixWrite
# - reuse stationsEnricheddf already loaded externally
# - DataFrames: stationsCountriesdf, countryStationCountsdf,
#   stateStationCountsdf, coreElementCountsdf, nzStationDistancesdf

#  

hprintf("Visualisation – Load DataFrames")

# paths
pqtStationsCountries     = f"{prefixWrite}/stations_countries.parquet"
pqtCountryCounts         = f"{prefixWrite}/country_station_counts.parquet"
pqtStateCounts           = f"{prefixWrite}/state_station_counts.parquet"
pqtCoreCounts            = f"{prefixWrite}/core_element_counts.parquet"
pqtNZDistances           = f"{prefixWrite}/nz_station_distances.parquet"


if not 0:
    start = time.time()

    stationsCountriesdf     = spark.read.parquet(pqtStationsCountries)
    countryStationCountsdf  = spark.read.parquet(pqtCountryCounts)
    stateStationCountsdf    = spark.read.parquet(pqtStateCounts)
    coreElementCountsdf     = spark.read.parquet(pqtCoreCounts)
    nzStationDistancesdf    = spark.read.parquet(pqtNZDistances)

    stop = time.time()
    dprintf(1, f"Read completed in {stop - start:.2f} seconds")
    dprintf(1, f"stationsCountriesdf.count()     = {stationsCountriesdf.count()}")
    dprintf(1, f"countryStationCountsdf.count()  = {countryStationCountsdf.count()}")
    dprintf(1, f"stateStationCountsdf.count()    = {stateStationCountsdf.count()}")
    dprintf(1, f"coreElementCountsdf.count()     = {coreElementCountsdf.count()}")
    dprintf(1, f"nzStationDistancesdf.count()    = {nzStationDistancesdf.count()}")
else:
    dprintf(1, f"DEEBUG = 1, no read from parquet")


In [ ]:
# CELL 2 Q1(a): – Global station density map | Summary: plot station locations from stationsCountriesdf as a LAT/LONG scatter

dprintf(True, "Q1(a): Global station distribution (scatter)")

import matplotlib.pyplot as plt

stationsPDF = stationsCountriesdf.select("LATITUDE", "LONGITUDE").toPandas()

fig, ax = plt.subplots(figsize=(12, 6))
ax.scatter(
    stationsPDF["LONGITUDE"], stationsPDF["LATITUDE"],
    s=1, alpha=0.5
)
ax.set_title("Global GHCN Station Locations", fontsize=14)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.grid(True)

plt.tight_layout()
plt.show()

dprintf(1, f"Total stations plotted = {len(stationsPDF)}")


In [ ]:
# CELL 3 Q1(b): – Network membership bar chart | Summary: count stations in GSN, HCN, and CRN networks from stationsEnricheddf

dprintf(True, "Q1(b): Network membership (GSN, HCN, CRN)")

import matplotlib.pyplot as plt

# create binary flags
networkCounts = {
    "GSN": stationsEnricheddf.filter(col("GSN_FLAG") != "").select("ID").distinct().count(),
    "HCN": stationsEnricheddf.filter(col("HCN_CRN_FLAG") == "H").select("ID").distinct().count(),
    "CRN": stationsEnricheddf.filter(col("HCN_CRN_FLAG") == "C").select("ID").distinct().count()
}

# convert to pandas-friendly format
import pandas as pd

networkdf = pd.DataFrame.from_dict(networkCounts, orient="index", columns=["nStations"])
networkdf = networkdf.reset_index().rename(columns={"index": "Network"})

# plot
fig, ax = plt.subplots(figsize=(8, 5))
ax.bar(networkdf["Network"], networkdf["nStations"], color="skyblue")
ax.set_title("Station Counts by Network (GSN, HCN, CRN)", fontsize=14)
ax.set_ylabel("Number of Stations")
ax.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

dprintf(1, f"GSN = {networkCounts['GSN']:,}, HCN = {networkCounts['HCN']:,}, CRN = {networkCounts['CRN']:,}")


In [ ]:
# CELL 4 Q1(c): – Stations per country (top 20) | Summary: plot horizontal bar chart using countryStationCountsdf

dprintf(True, "Q1(c): Stations per country (top 20)")

import matplotlib.pyplot as plt

# convert to pandas for plotting
countryTopNpdf = countryStationCountsdf.orderBy(col("nStations").desc()).limit(20).toPandas()

# plot
fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(countryTopNpdf["COUNTRY_NAME"], countryTopNpdf["nStations"], color="steelblue")
ax.invert_yaxis()  # largest at top
ax.set_xlabel("Number of Stations")
ax.set_title("Top 20 Countries by Station Count", fontsize=14)
ax.grid(axis="x", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

dprintf(1, f"Max: {countryTopNpdf.iloc[0]['COUNTRY_NAME']} = {countryTopNpdf.iloc[0]['nStations']:,} stations")


In [ ]:
# CELL 5 Q1(d): – Map of stations active in 2025 | Summary: filter stationsEnricheddf by FIRSTYEAR/LASTYEAR and plot LAT/LONG

dprintf(True, "Q1(d): Map of stations active in 2025")

import matplotlib.pyplot as plt

# filter by activity window including 2025
stations2025df = stationsEnricheddf.filter(
    (col("FIRSTYEAR") <= 2025) & (col("LASTYEAR") >= 2025)
)

# convert to pandas for plotting
stations2025pdf = stations2025df.select("LATITUDE", "LONGITUDE").toPandas()

# plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.scatter(
    stations2025pdf["LONGITUDE"], stations2025pdf["LATITUDE"],
    s=1, alpha=0.5, color="orange"
)
ax.set_title("Stations Active in 2025", fontsize=14)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.grid(True)

plt.tight_layout()
plt.show()

dprintf(1, f"Active stations in 2025 = {len(stations2025pdf):,}")


In [ ]:
# CELL 6 Q2(a): – Histogram of NZ station distances | Summary: plot distribution of pairwise distances from nzStationDistancesdf

dprintf(True, "Q2(a): Histogram of pairwise NZ station distances")

import matplotlib.pyplot as plt

# convert to pandas
nzDistancesPDF = nzStationDistancesdf.select("DistanceKM").toPandas()

# plot histogram
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(nzDistancesPDF["DistanceKM"], bins=30, color="seagreen", edgecolor="black")
ax.set_title("Pairwise Distances Between NZ Stations", fontsize=14)
ax.set_xlabel("Distance (km)")
ax.set_ylabel("Number of Pairs")
ax.grid(True)

plt.tight_layout()
plt.show()

dprintf(1, f"Total station pairs (NZ) = {len(nzDistancesPDF):,}")


In [ ]:
# CELL 7 Q2(b): – Closest NZ station pair map
# CELL 7 Q2(b): 
# ------------------------------------------------
# Summary:
# - extract closest pair of stations from nzStationDistancesdf
# - retrieve lat/lon from stationsCountriesdf
# - plot a map with a red line between the closest pair
# - label each station and print distance in km

#  

dprintf(True, "Q2(b): Closest NZ station pair on map")

import matplotlib.pyplot as plt

# extract closest pair (minimum DistanceKM)
closestPair = nzStationDistancesdf.orderBy("DistanceKM").limit(1).collect()[0]

# extract station details
id1 = closestPair["ID1"]
id2 = closestPair["ID2"]
name1 = closestPair["Name1"]
name2 = closestPair["Name2"]
lat1 = stationsCountriesdf.filter(col("ID") == id1).select("LATITUDE").collect()[0][0]
lon1 = stationsCountriesdf.filter(col("ID") == id1).select("LONGITUDE").collect()[0][0]
lat2 = stationsCountriesdf.filter(col("ID") == id2).select("LATITUDE").collect()[0][0]
lon2 = stationsCountriesdf.filter(col("ID") == id2).select("LONGITUDE").collect()[0][0]

# plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot([lon1, lon2], [lat1, lat2], 'r-', linewidth=2, label="Closest pair")
ax.scatter([lon1, lon2], [lat1, lat2], color='blue', zorder=5)

ax.text(lon1, lat1, name1, fontsize=9, ha="right", va="bottom")
ax.text(lon2, lat2, name2, fontsize=9, ha="left", va="bottom")

ax.set_title("Closest Pair of NZ Stations", fontsize=14)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.grid(True)
ax.legend()

plt.tight_layout()
plt.show()

dprintf(1, f"Closest pair: {name1} ↔ {name2} ({closestPair['DistanceKM']:.2f} km)")


In [ ]:
# CELL 8 Q2(c): – Bar chart of core element observation counts
# CELL 8 Q2(c): 
# ------------------------------------------------
# Summary:
# - visualise counts of core elements (PRCP, TMAX, etc.) from coreElementCountsdf
# - convert to pandas and plot with matplotlib
# - output vertical bar chart

#  

dprintf(True, "Q2(c): Observation counts by core element")

coreCountsPDF = coreElementCountsdf.orderBy(col("count").desc()).toPandas()

fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(coreCountsPDF["ELEMENT"], coreCountsPDF["count"], color="mediumslateblue")
ax.set_title("Observation Counts by Core Element", fontsize=14)
ax.set_ylabel("Number of Observations")
ax.set_xlabel("Element")
ax.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

dprintf(1, f"Most observed element: {coreCountsPDF.iloc[0]['ELEMENT']} ({coreCountsPDF.iloc[0]['count']:,} observations)")


In [ ]:
# CELL 9 Q2(d): – Pie chart of core element share
# CELL 9 Q2(d): 
# ------------------------------------------------
# Summary:
# - visualise proportion of observations by element using coreElementCountsdf
# - convert to pandas and generate pie chart with matplotlib
# - annotate chart with percentages

#  

dprintf(True, "Q2(d): Pie chart of core element observation proportions")

coreCountsPDF = coreElementCountsdf.orderBy(col("count").desc()).toPandas()

fig, ax = plt.subplots(figsize=(8, 6))
ax.pie(coreCountsPDF["count"], labels=coreCountsPDF["ELEMENT"], autopct='%1.1f%%', startangle=90)
ax.set_title("Proportion of Observations by Core Element", fontsize=14)

plt.tight_layout()
plt.show()

dprintf(1, "Pie chart rendered for core elements")
